In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import load_model

In [1]:
import numpy as np
import pandas as pd
import os

In [ ]:
#設定LSTM往前看的筆數和預測筆數
LookBackNum = 12 #LSTM往前看的筆數
ForecastNum = 48 #預測筆數

SourceData_08 = pd.read_csv('/Users/chiao/datascience_hwc/cp2/AvgDATA_08.csv', encoding='utf-8')
SourceData_03 = pd.read_csv('/Users/chiao/datascience_hwc/cp2/AvgDATA_03.csv', encoding='utf-8')

# 合併兩個 DataFrame
SourceData = pd.concat([SourceData_08, SourceData_03], axis=0, ignore_index=True)

# 選擇要留下來的資料欄位 (發電量)
target = ['Power(mW)']
AllOutPut = SourceData[target].values

X_train = []
y_train = []

#設定每i-12筆資料(X_train)就對應到第i筆資料(y_train)
for i in range(LookBackNum,len(AllOutPut)):
  X_train.append(AllOutPut[i-LookBackNum:i, 0])
  y_train.append(AllOutPut[i, 0])

X_train = np.array(X_train)
y_train = np.array(y_train)

# Reshaping
X_train = np.reshape(X_train,(X_train.shape [0], X_train.shape [1], 1))

In [ ]:
regressor = Sequential ()

regressor.add(LSTM(units = 128, return_sequences = True, input_shape = (X_train.shape[1], 1)))

regressor.add(LSTM(units = 64))

regressor.add(Dropout(0.2))

# output layer
regressor.add(Dense(units = 1))
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

#開始訓練
regressor.fit(X_train, y_train, epochs = 100, batch_size = 128)

#保存模型
from datetime import datetime
NowDateTime = datetime.now().strftime("%Y-%m-%dT%H_%M_%SZ")
regressor.save('WheatherLSTM_'+NowDateTime+'.h5')
print('Model Saved')

In [ ]:
#載入模型
regressor = load_model('WheatherLSTM_2024-11-27T23_14_11Z.h5')

#載入測試資料
SourceData = pd.read_csv('/Users/chiao/datascience_hwc/cp2/upload(no answer).csv', encoding='utf-8')
target = ['序號']
EXquestion = SourceData[target].values

inputs = []#存放參考資料
PredictOutput = [] #存放預測值

count = 0
while(count < len(EXquestion)):
  print('count : ',count)
  LocationCode = int(EXquestion[count])
  strLocationCode = str(LocationCode)[-2:]
  if LocationCode < 10 :
    strLocationCode = '0'+LocationCode

  SourceData = pd.read_csv('/Users/chiao/datascience_hwc/cp2/TrainingData(IncompleteAVG)/IncompleteAvgDATA_'+ strLocationCode +'.csv', encoding='utf-8')
  ReferTitle = SourceData[['Serial']].values
  ReferData = SourceData[['Power(mW)']].values
  
  inputs = []#重置存放參考資料

  #找到相同的一天，把12個資料都加進inputs
  for DaysCount in range(len(ReferTitle)):
    if(str(int(ReferTitle[DaysCount]))[:8] == str(int(EXquestion[count]))[:8]):
      inputs = np.append(inputs, ReferData[DaysCount])
         
  #用迴圈不斷使新的預測值塞入參考資料，並預測下一筆資料
  for i in range(ForecastNum) :

    #print(i)
    
    #將新的預測值加入參考資料(用自己的預測值往前看)
    if i > 0 :
      inputs = np.append(inputs, PredictOutput[i-1])
    
    #切出新的參考資料12筆(往前看12筆)
    X_test = []
    X_test.append(inputs[0+i:LookBackNum+i])
    
    #Reshaping
    NewTest = np.array(X_test)
    NewTest = np.reshape(NewTest, (NewTest.shape[0], NewTest.shape[1], 1))
    predicted = regressor.predict(NewTest)
    PredictOutput.append(round(predicted[0,0], 2))
  
  #每次預測都要預測48個，因此加48個會切到下一天
  #0~47,48~95,96~143...
  count += 48

#寫預測結果寫成新的CSV檔案
# 將陣列轉換為 DataFrame
df = pd.DataFrame(PredictOutput, columns=['答案'])

# 將 DataFrame 寫入 CSV 檔案
df.to_csv('output.csv', index=False) 
print('Output CSV File Saved')
